In [1]:
import pandas as pd
import json
import os

In [2]:
years = range(2000, 2019)

In [3]:
def get_iso(weo_filepath):
    weo = pd.read_csv(weo_filepath)

    columns_to_check = [str(year) for year in range(2000, 2023)]
    countries_with_missing_data = weo[weo[columns_to_check].isna().any(axis=1)][
        "ISO"
    ].unique()
    weo_cleaned = weo[~weo["ISO"].isin(countries_with_missing_data)]
    all_iso = weo_cleaned["ISO"].unique().tolist()
    iso_to_name = dict(zip(weo_cleaned["ISO"], weo_cleaned["Country"]))
    return all_iso, iso_to_name


weo_filepath = "./WEO_Data_Full.csv"
selected_iso, iso_to_name = get_iso(weo_filepath)
selected_iso


['ALB',
 'DZA',
 'AGO',
 'ATG',
 'ARG',
 'ARM',
 'ABW',
 'AUS',
 'AUT',
 'AZE',
 'BHS',
 'BHR',
 'BGD',
 'BRB',
 'BLR',
 'BEL',
 'BLZ',
 'BEN',
 'BTN',
 'BOL',
 'BIH',
 'BWA',
 'BRA',
 'BRN',
 'BGR',
 'BFA',
 'BDI',
 'CPV',
 'KHM',
 'CMR',
 'CAN',
 'CAF',
 'TCD',
 'CHL',
 'CHN',
 'COL',
 'COM',
 'COD',
 'COG',
 'CRI',
 'CIV',
 'HRV',
 'CYP',
 'CZE',
 'DNK',
 'DJI',
 'DMA',
 'DOM',
 'ECU',
 'EGY',
 'SLV',
 'GNQ',
 'ERI',
 'EST',
 'SWZ',
 'ETH',
 'FJI',
 'FIN',
 'FRA',
 'GAB',
 'GMB',
 'GEO',
 'DEU',
 'GHA',
 'GRC',
 'GRD',
 'GTM',
 'GIN',
 'GNB',
 'GUY',
 'HTI',
 'HND',
 'HKG',
 'HUN',
 'ISL',
 'IND',
 'IDN',
 'IRN',
 'IRL',
 'ISR',
 'ITA',
 'JAM',
 'JPN',
 'JOR',
 'KAZ',
 'KEN',
 'KIR',
 'KOR',
 'KWT',
 'KGZ',
 'LAO',
 'LVA',
 'LSO',
 'LBY',
 'LTU',
 'LUX',
 'MDG',
 'MWI',
 'MYS',
 'MDV',
 'MLI',
 'MHL',
 'MRT',
 'MUS',
 'MEX',
 'FSM',
 'MDA',
 'MNG',
 'MAR',
 'MOZ',
 'MMR',
 'NAM',
 'NPL',
 'NLD',
 'NZL',
 'NIC',
 'NER',
 'NGA',
 'MKD',
 'NOR',
 'OMN',
 'PAK',
 'PAN',
 'PNG',
 'PRY',


In [4]:
iso_to_name_json = json.dumps(iso_to_name, indent=2)
output_filepath = "isoFullName.json"
with open(output_filepath, "w") as file:
    file.write(iso_to_name_json)

In [5]:
def generate_color_mapping(countries, color_format="#{:06x}"):
    def get_color(index):
        return color_format.format(index * 123457 % 0xFFFFFF)

    return {country: get_color(index) for index, country in enumerate(countries)}


country_colors = generate_color_mapping(selected_iso)
country_colors_json = json.dumps(country_colors, indent=2)
country_colors_json
output_filepath = "countryColors"
with open(output_filepath, "w") as file:
    file.write(country_colors_json)

In [6]:
country_data = {iso: {} for iso in selected_iso}

for iso in selected_iso:
    for year in years:
        country_data[iso][year] = {}

country_data

{'ALB': {2000: {},
  2001: {},
  2002: {},
  2003: {},
  2004: {},
  2005: {},
  2006: {},
  2007: {},
  2008: {},
  2009: {},
  2010: {},
  2011: {},
  2012: {},
  2013: {},
  2014: {},
  2015: {},
  2016: {},
  2017: {},
  2018: {}},
 'DZA': {2000: {},
  2001: {},
  2002: {},
  2003: {},
  2004: {},
  2005: {},
  2006: {},
  2007: {},
  2008: {},
  2009: {},
  2010: {},
  2011: {},
  2012: {},
  2013: {},
  2014: {},
  2015: {},
  2016: {},
  2017: {},
  2018: {}},
 'AGO': {2000: {},
  2001: {},
  2002: {},
  2003: {},
  2004: {},
  2005: {},
  2006: {},
  2007: {},
  2008: {},
  2009: {},
  2010: {},
  2011: {},
  2012: {},
  2013: {},
  2014: {},
  2015: {},
  2016: {},
  2017: {},
  2018: {}},
 'ATG': {2000: {},
  2001: {},
  2002: {},
  2003: {},
  2004: {},
  2005: {},
  2006: {},
  2007: {},
  2008: {},
  2009: {},
  2010: {},
  2011: {},
  2012: {},
  2013: {},
  2014: {},
  2015: {},
  2016: {},
  2017: {},
  2018: {}},
 'ARG': {2000: {},
  2001: {},
  2002: {},
  2003: {},
 

In [7]:
def processIndicator(ind_filepath, selected_iso):
    base_name = os.path.basename(ind_filepath)
    ind_name = base_name.split("_")[0]
    df = pd.read_csv(ind_filepath)

    # Drop iso and keep country code for joining purposes in the future
    df.drop(["code", "country"], axis=1, inplace=True)
    df = df.fillna(0)

    # Extract year info from original column names - "CDA.raw.YYYY"
    df.columns = ["iso"] + [col.split(".")[-1] for col in df.columns[1:]]

    filtered_df = df[df["iso"].isin(selected_iso)]

    for _, row in filtered_df.iterrows():
        iso = row["iso"]
        for year in years:
            if str(year) in row:
                country_data[iso][year][ind_name] = row[str(year)]
            else:
                country_data[iso][year][ind_name] = 0

In [8]:
def processAllIndicator(folder_path, selected_iso):
    for filename in os.listdir(folder_path):
        if filename.endswith("_raw_na.csv"):
            full_path = os.path.join(folder_path, filename)
            processIndicator(full_path, selected_iso)


folder_path = "./"

processAllIndicator(folder_path, selected_iso)

In [9]:
def processWEO(filename, selected_iso):
    weo_raw = pd.read_csv(filename)
    filtered_weo = (
        weo_raw
        # Filter out records with unit of national currency. Keep records with unit of U.S. dollars for meaningful comparison.
        .query("Units == 'U.S. dollars'")
        .drop(
            [
                "Country",
                "Estimates Start After",
                "Units",
            ],
            axis=1,
        )
        .rename(
            columns={
                "ISO": "iso",
                "Subject Descriptor": "type",
            }
        )
        .replace(
            {
                "Gross domestic product, current prices": "GDP",
                "Gross domestic product per capita, current prices": "GDPPC",
            }
        )
    )

    filtered_weo = filtered_weo[filtered_weo["iso"].isin(selected_iso)]
    # Unify the currency scale
    # filtered_weo.loc[
    #     filtered_weo["Scale"] == "Billions", filtered_weo.columns[4:]
    # ] *= 1000
    filtered_weo = filtered_weo.drop("Scale", axis=1)

    for _, row in filtered_weo.iterrows():
        iso = row["iso"]
        for year in years:
            if str(year) in row:
                if row["type"] == "GDP":
                    country_data[iso][year]["GDP"] = row[str(year)]
                else:
                    country_data[iso][year]["GDPPC"] = row[str(year)]
            else:
                if row["type"] == "GDP":
                    country_data[iso][year]["GDP"] = 0
                else:
                    country_data[iso][year]["GDPPC"] = 0



processWEO("./WEO_Data_Full.csv", selected_iso)

country_data

{'ALB': {2000: {'CDA': -0.0314404754663873,
   'NOE': 0,
   'VOE': 0,
   'HAD': 1629.89603944769,
   'COE': 0,
   'NDA': 0.0067293883024568,
   'PMD': 1039.72748146333,
   'OZD': 26.1955211342509,
   'SOE': 0,
   'GDP': '3.48',
   'GDPPC': '1127.64'},
  2001: {'CDA': 0.0284847036278371,
   'NOE': 0,
   'VOE': 0,
   'HAD': 1453.72289198402,
   'COE': 0,
   'NDA': 0.0029628801785903,
   'PMD': 988.963686572596,
   'OZD': 24.4738158468714,
   'SOE': 0,
   'GDP': '3.93',
   'GDPPC': '1283.57'},
  2002: {'CDA': 0.0622242656974739,
   'NOE': 0,
   'VOE': 0,
   'HAD': 1381.82378655363,
   'COE': 0,
   'NDA': 0.0008567775780923,
   'PMD': 1004.53424215074,
   'OZD': 28.5938638194365,
   'SOE': 0,
   'GDP': '4.35',
   'GDPPC': '1425.13'},
  2003: {'CDA': 0.0947645206166079,
   'NOE': 0.011609208,
   'VOE': 0.009877494,
   'HAD': 1334.19312778145,
   'COE': 0.196819358,
   'NDA': 0.0006234796894228,
   'PMD': 1041.91346385018,
   'OZD': 32.3319069089824,
   'SOE': 0.016917499,
   'GDP': '5.61',


In [10]:
weo_raw = pd.read_csv("./WEO_Data_Full.csv")
filtered_weo = (
    weo_raw
    # Filter out records with unit of national currency. Keep records with unit of U.S. dollars for meaningful comparison.
    .query("Units == 'U.S. dollars'")
    .drop(
        [
            "Country",
            "Estimates Start After",
            "Units",
        ],
        axis=1,
    )
    .rename(
        columns={
            "ISO": "iso",
            "Subject Descriptor": "type",
        }
    )
    .replace(
        {
            "Gross domestic product, current prices": "GDP",
            "Gross domestic product per capita, current prices": "GDPPC",
        }
    )
)
filtered_weo = filtered_weo[filtered_weo["iso"].isin(selected_iso)]
filtered_weo.loc[filtered_weo["Scale"] == "Millions", filtered_weo.columns[3:]]
filtered_weo = filtered_weo.drop("Scale", axis=1)

filtered_weo

,iso,type,2000,2001,2002,2003,2004,2005,2006,2007,...,2016,2017,2018,2019,2020,2021,2022,Unnamed: 29,Unnamed: 30,Unnamed: 31
8,ALB,GDP,3.48,3.93,4.35,5.61,7.19,8.05,8.90,10.68,...,11.86,13.05,15.16,15.40,15.16,18.31,18.51,NaN,NaN,NaN
10,ALB,GDPPC,1127.64,1283.57,1425.13,1846.12,2373.58,2673.77,2972.75,3595.05,...,4124.41,4525.89,5257.71,5345.06,5268.24,6373.30,6457.22,NaN,NaN,NaN
14,DZA,GDP,54.75,54.75,56.76,67.86,85.33,103.20,117.03,134.98,...,160.03,167.50,174.87,171.67,145.66,163.14,195.42,NaN,NaN,NaN
16,DZA,GDPPC,1794.70,1768.58,1806.86,2128.39,2636.48,3141.03,3508.96,3986.56,...,3918.94,4014.71,4107.00,3953.40,3321.60,3659.71,4314.66,NaN,NaN,NaN
26,AGO,GDP,11.17,10.93,15.29,17.81,23.55,36.97,52.38,65.27,...,101.12,122.02,101.35,84.52,57.14,74.79,121.42,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1162,YEM,GDPPC,526.88,521.70,551.19,591.22,677.90,796.36,881.28,970.82,...,1083.89,905.18,710.38,702.29,575.46,519.49,619.55,NaN,NaN,NaN
1166,ZMB,GDP,3.60,3.87,4.19,4.90,6.22,8.33,12.76,14.06,...,20.97,25.87,26.31,23.31,18.11,22.15,28.50,NaN,NaN,NaN
1168,ZMB,GDPPC,364.03,379.71,399.10,452.29,556.05,720.20,1066.02,1133.67,...,1250.32,1495.75,1475.20,1268.12,956.83,1137.34,1423.75,NaN,NaN,NaN
1172,ZWE,GDP,11.34,11.24,10.74,9.57,9.47,9.05,8.14,7.79,...,20.56,21.39,36.95,26.03,26.88,35.97,33.02,NaN,NaN,NaN


In [11]:
def processPopulation(pop_filepath, selected_iso):
    pop = pd.read_csv(pop_filepath)

    filtered_pop = (
        pop.query('`Subject Descriptor` == "Population"')
        .drop(
            [
                "Country",
    

                "Estimates Start After",
                "Units",
                "Scale",
                "Subject Descriptor",
            ],
            axis=1,
        )
        .rename(
            columns={
                "ISO": "iso",
            }
        )
    )

    filtered_pop = filtered_pop[filtered_pop["iso"].isin(selected_iso)]
    for _, row in filtered_pop.iterrows():
        iso = row["iso"]
        for year in years:
            if str(year) in row:
                country_data[iso][year]["Population"] = row[str(year)]


processPopulation("./POP_Data_Full.csv", selected_iso)

In [12]:
country_data

{'ALB': {2000: {'CDA': -0.0314404754663873,
   'NOE': 0,
   'VOE': 0,
   'HAD': 1629.89603944769,
   'COE': 0,
   'NDA': 0.0067293883024568,
   'PMD': 1039.72748146333,
   'OZD': 26.1955211342509,
   'SOE': 0,
   'GDP': '3.48',
   'GDPPC': '1127.64',
   'Population': '3.09'},
  2001: {'CDA': 0.0284847036278371,
   'NOE': 0,
   'VOE': 0,
   'HAD': 1453.72289198402,
   'COE': 0,
   'NDA': 0.0029628801785903,
   'PMD': 988.963686572596,
   'OZD': 24.4738158468714,
   'SOE': 0,
   'GDP': '3.93',
   'GDPPC': '1283.57',
   'Population': '3.06'},
  2002: {'CDA': 0.0622242656974739,
   'NOE': 0,
   'VOE': 0,
   'HAD': 1381.82378655363,
   'COE': 0,
   'NDA': 0.0008567775780923,
   'PMD': 1004.53424215074,
   'OZD': 28.5938638194365,
   'SOE': 0,
   'GDP': '4.35',
   'GDPPC': '1425.13',
   'Population': '3.05'},
  2003: {'CDA': 0.0947645206166079,
   'NOE': 0.011609208,
   'VOE': 0.009877494,
   'HAD': 1334.19312778145,
   'COE': 0.196819358,
   'NDA': 0.0006234796894228,
   'PMD': 1041.9134638

In [13]:
country_data_json = json.dumps(country_data, indent=4)
output_filepath = "country_data_full.json"
with open(output_filepath, "w") as file:
    file.write(country_data_json)